# Day 4 notebook

The objectives of this notebook are to practice

* the concept of read overlap and merging
* finding solutions to the shortest superstring task
* implementing graph data structures
* defining Python classes
* using inheritance with Python classes

## The shortest superstring problem

### PROBLEM 1: Finding the maximum overlap length of two strings (reads) (1 POINT)

A key component of most algorithms for addressing the shortest superstring problem is determining the length of the longest exact overlap between two strings.  In particular, for two strings, $left$ and $right$, we need to compute $overlap\_length(left, right)$, which is defined as the length of the longest **suffix** of $left$ that is a **prefix** of $right$.  Define a function that computes this value.  You may find the [endswith](https://docs.python.org/3/library/stdtypes.html#str.endswith) or [startswith](https://docs.python.org/3/library/stdtypes.html#str.startswith) methods of Python string objects helpful.  You do not need to implement the most efficient algorithm possible (a simple polynomial-time algorithm will suffice).

In [1]:
def overlap_length(left, right):
    """Returns the length of the longest suffix of left that is a prefix of right
    
    Args:
        left: a string
        right: a string
    Returns:
        An integer length of the longest overlap (0 if there is no overlap)
    """
    ### BEGIN SOLUTION
    for length in range(min(len(left), len(right)), 0, -1):
        if left.endswith(right[:length]):
            return length
    else:
        return 0
    ### END SOLUTION

In [2]:
# tests for overlap_length
assert overlap_length("ATGC", "GCAT") == 2
assert overlap_length("AGCA", "AGCA") == 4
assert overlap_length("AGAG", "GAGT") == 3
assert overlap_length("GAGT", "AGAG") == 0
assert overlap_length("ATGC", "CA") == 1
assert overlap_length("CA", "ATCG") == 1
assert overlap_length("", "") == 0
print("SUCCESS: overlap_length passed all tests!")

SUCCESS: overlap_length passed all tests!


### PROBLEM 2: Merging ordered reads (1 POINT)
One way to think about constructing a superstring for a set of strings (reads) is to note that any given superstring imposes an ordering on the input strings, namely, the order in which those strings appear as substrings within the superstring.  With the assumption that no input string is a substring of any other input string, for any fixed ordering of of the input strings, there is a unique shortest superstring that imposes that ordering.  That superstring may be constructed by simply merging strings from left to right in the ordering, using the maximum possible overlap between adjacent strings.  Write a function below that computes the shortest superstring for a given ordered list of strings (reads).  You will likely need your `overlap_length` function from above.

In [3]:
def merge_ordered_reads(reads):
    """Returns the shortest superstring resulting from
    merging a list of ordered reads.
    
    Args:
        reads: a list of strings
    Returns:
        A string that is a shortest superstring of the ordered input read strings.
    """

    ### BEGIN SOLUTION
    read_pieces = []
    prev_read = ""
    for read in reads:
        overlap_with_prev_read = overlap_length(prev_read, read)
        read_pieces.append(read[overlap_with_prev_read:])
        prev_read = read
    return ''.join(read_pieces)
    ### END SOLUTION

In [4]:
# tests for merge_ordered_reads
assert merge_ordered_reads(["AGAG", "GAGT"]) == "AGAGT"
assert merge_ordered_reads(["AGAG"]) == "AGAG"
assert merge_ordered_reads(["AGAG", "GAGT", "AGTC"]) == "AGAGTC"
assert merge_ordered_reads(["AGAG", "CGAG", "TCGA"]) == "AGAGCGAGTCGA"
assert merge_ordered_reads([]) == ""
print("SUCCESS: merge_ordered_reads passed all tests!")

SUCCESS: merge_ordered_reads passed all tests!


### PROBLEM 3: Brute-force shortest superstring algorithm (1 POINT)
Continuing with the reasoning from the previous problem, note that a shortest superstring for a set of (unordered) strings corresponds to some ordering of the strings.  So if we try every possible ordering of the strings and compute the shortest superstring for each ordering (using the `merge_ordered_reads` function above), we could find a solution to the shortest superstring problem.  Write a function `shortest_superstring` below, which implements this "brute-force" strategy for solving the shortest superstring problem given a set of input strings (reads).

You will likely find the [`permutations`](https://docs.python.org/3/library/itertools.html#itertools.permutations) function from the [`itertools`](https://docs.python.org/3/library/itertools.html) module in the Python standard library of use.  This function returns an iterator over all possible orderings (permutations) of the sequence given to it as input.

In the case that there are multiple shortest superstrings, your function should return the superstring that is lexicographically smallest (i.e., first in alphabetical order).

In [5]:
import itertools

def shortest_superstring(reads):
    """Returns the shortest superstring of a set of reads.

    Assumes that no string in the input is a substring of another input string.
    
    Args:
        reads: a list of strings
    Returns:
        A string that is a shortest superstring of reads.  In the case
        of multiple shortest superstrings, the lexicographically
        smallest is returned.
    """
    ### BEGIN SOLUTION
    s = None
    for read_permutation in itertools.permutations(reads):
        merged = merge_ordered_reads(read_permutation)
        if (s is None or 
            len(merged) < len(s) or
            (len(merged) == len(s) and merged < s)):
            s = merged
    return s
    ### END SOLUTION

In [6]:
# tests for shortest_superstring
assert shortest_superstring(['AGAT', 'CAGA', 'ATAT']) == "CAGATAT"
assert shortest_superstring(['GACC', 'ATCC', 'TACC']) == "ATCCGACCTACC"
assert shortest_superstring(['TGA', 'GAC', 'ATGC']) == "ATGCTGAC"
assert shortest_superstring(['TGA']) == "TGA"
assert shortest_superstring(['TGA', 'CTG']) == "CTGA"
print("SUCCESS: shortest_superstring passed all tests!")

SUCCESS: shortest_superstring passed all tests!


### Thinking about the brute-force shortest superstring algorithm
Assuming you have written it correctly, the above function solves the shortest superstring problem.  Is the algorithm implemented by this function efficient?  In the cell below, write down your thoughts regarding the computational complexity of this algorithm.

### BEGIN SOLUTION TEMPLATE=YOUR THOUGHTS HERE REGARDING THE COMPUTATIONAL COMPLEXITY OF THE ALGORITHM YOU IMPLEMENTED ABOVE
This algorithm is far from efficient.  It must iterate over all $n!$ possible orderings of the $n$ reads, which grows faster than an expoential function, for large $n$.
### END SOLUTION

### Experimenting with your shortest superstring algorithm
If you have time, experiment with running your brute-force shortest superstring algorithm on various inputs.  For example,

1. Experiment with inputs of random DNA strings (I've provided some code for you below to generate such strings).  How short can the shortest superstring be given random inputs, relative to the the trivial superstring that simply concatenates the input strings?  Perhaps try with many random inputs and plot the distribution of the shortest superstring length, say with the [hist](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function of Pyplot.

2. Experiment with (very small sets) of strings generated by your shotgun sequencing simulator in the Day 3 notebook.  Does the shortest superstring reconstruct the full DNA sequence?  

In [7]:
# Some functions that you may find of use in experimenting with random inputs of DNA strings
import random
DNA_BASES = "ACGT"

def random_dna_string(length):
    """Returns a random DNA string of the given length"""
    return ''.join([random.choice(DNA_BASES) for i in range(length)])

def random_dna_strings(length, num_strings):
    """Returns a list of num_strings random DNA strings of the given length"""
    return [random_dna_string(length) for i in range(num_strings)]

*YOUR EXPERIMENTS HERE*

## Implementing graphs as Python classes
Many sequence assembly algorithms are formulated in terms of *graphs*.  In this section of the notebook, we will define a Python class that implements a graph data structure.

### A base class for directed graphs

There are many possible ways to implement a graph data structure and each has its own strengths and weaknesses in terms of memory usage and efficiency of various graph operations.  Algorithms that use graphs do not necessarily need to know how those graphs are implemented, and therefore it is helpful to separate the *interface* of a graph object from its underlying *implementation*.  Classes are a great way to provide such abstractions.

Below I have provided a *base* class for a directed graph, which specifies the interface that a general directed graph object should have.  This is an *abstract base class*, which is only meant to provide an interface and some default method definitions.  It is not meant to be instantiated directly.  Note that the key methods of `had_edge` and `add_edge` are not implemented.

With this directed graph class, we simply refer to each vertex by an integer index (0 to `num_vertices - 1`), and an edge *from* vertex `i` *to* vertex `j` is represented by the tuple `(i, j)`.

Study the class definition below to understand the inferface and the default implementations.

In [8]:
class DirectedGraph:
    """Abstract base class for a directed graph.

    A functional directed graph class can be obtained by inheriting from 
    this class and overriding the methods has_edge and add_edge.  All other
    methods have default implementations, which may not be the most efficient.
    These other methods should also be overriden as appropriate to improve
    efficiency.
    """
    def __init__(self, num_vertices):
        """Constructs a directed graph with num_vertices vertices and zero edges"""
        self._num_vertices = num_vertices
    
    def has_edge(self, i, j):
        """Returns True if the graph contains the directed edge (i, j), False otherwise."""
        raise NotImplementedError
        
    def add_edge(self, i, j):
        """Adds the directed edge (i, j) to the graph."""
        raise NotImplementedError
        
    def out_edges(self, i):
        """Returns a list of directed edges outgoing from vertex i."""
        return [(i, j) for j in range(self._num_vertices) if self.has_edge(i, j)]
    
    def in_edges(self, j):
        """Returns a list of directed edges incoming to vertex j."""
        return [(i, j) for i in range(self._num_vertices) if self.has_edge(i, j)]
    
    def outdegree(self, i):
        """Returns the outdegree of vertex i."""
        return len(self.out_edges(i))
    
    def indegree(self, i):
        """Returns the indegree of vertex i."""
        return len(self.in_edges(i))
    
    def degree(self, i):
        """Returns the degree of vertex i."""
        return self.indegree(i) + self.outdegree(i)
        
    def add_edges(self, edges):
        """Adds all edges from a list to the graph."""
        for i, j in edges:
            self.add_edge(i, j)
            
    def num_vertices(self):
        """Returns the number of vertices in the graph."""
        return self._num_vertices

    def num_edges(self):
        """Returns the number of edges in the graph."""
        return len(tuple(self.edges()))
    
    def edges(self):
        """Returns an iterator over the edges of the graph."""
        for i in range(self._num_vertices):
            for edge in self.out_edges(i):
                yield edge
    
    def __str__(self):
        """Returns a string representation of the graph, so that it may be printed."""
        return "DirectedGraph with %d vertices and %d edge(s):\n%s" % (self.num_vertices(),
                                                                       self.num_edges(),
                                                                       sorted(self.edges()))

### A trival functional DirectedGraph class
As an example of how to construct a fully-functional class that provides the DirectedGraph interface, below is a class that simply stores its edges in a set.  This results in relatively inefficient graph operations.  Examine the implementation below and how it *inherits* functionality from the base class `DirectedGraph`.

In [9]:
class TrivialSetDirectedGraph(DirectedGraph):
    def __init__(self, num_vertices):
        # call the parent class's constructor
        super().__init__(num_vertices)
        # start with an empty set of edges
        self._edges = set()
    
    def has_edge(self, i, j):
        return (i, j) in self._edges
        
    def add_edge(self, i, j):
        self._edges.add((i, j))

In [10]:
# An example of using TrivalSetDirectedGraph
my_graph = TrivialSetDirectedGraph(4)
print(my_graph)
my_graph.add_edge(3, 0)
print(my_graph)
my_graph.add_edges([(0, 2), (2, 3), (3, 1)])
print(my_graph)
print("Outdegree of vertex 3 =", my_graph.outdegree(3))
print("Graph has edge (2, 3):", my_graph.has_edge(2, 3))
print("Graph has edge (3, 2):", my_graph.has_edge(3, 2))

DirectedGraph with 4 vertices and 0 edge(s):
[]
DirectedGraph with 4 vertices and 1 edge(s):
[(3, 0)]
DirectedGraph with 4 vertices and 4 edge(s):
[(0, 2), (2, 3), (3, 0), (3, 1)]
Outdegree of vertex 3 = 2
Graph has edge (2, 3): True
Graph has edge (3, 2): False


### PROBLEM 4: Adjacency list implementation of a directed graph (4 POINTS)
A more efficent and common implementation strategy for graphs is to use *adjacency lists*.  In an adjacency list data structure, for each vertex, a list of vertices to which that vertex is connected by an edge is maintained.  For directed graphs, two lists per vertex can be used, one for all vertices that are adjacent via an *outgoing* edge and one for all vertices that are adjacent via an *incoming* edge.

With this implementation strategy, define the `AdjacencyListDirectedGraph` class below.  You will need to provide concrete definitions for the methods `add_edge` and `has_edge`.  You also need to override the constructor `__init__`, and methods `out_edges`, `in_edges`, `indegree`, and `outdegree` to make them more efficient with this implementation.

In [11]:
class AdjacencyListDirectedGraph(DirectedGraph):
    def __init__(self, num_vertices):
        ### BEGIN SOLUTION
        super().__init__(num_vertices)
        self._out_lists = [[] for i in range(num_vertices)]
        self._in_lists = [[] for i in range(num_vertices)]
        ### END SOLUTION
    
    def add_edge(self, i, j):
        ### BEGIN SOLUTION
        self._out_lists[i].append(j)
        self._in_lists[j].append(i)
        ### END SOLUTION
    
    def has_edge(self, i, j):
        ### BEGIN SOLUTION
        return j in self._out_lists[i]
        ### END SOLUTION
        
    def out_edges(self, i):
        ### BEGIN SOLUTION
        return [(i, j) for j in self._out_lists[i]]
        ### END SOLUTION
        
    def in_edges(self, j):
        ### BEGIN SOLUTION
        return [(i, j) for i in self._in_lists[j]]
        ### END SOLUTION
    
    def indegree(self, i):
        ### BEGIN SOLUTION
        return len(self._in_lists[i])
        ### END SOLUTION
        
    def outdegree(self, i):
        ### BEGIN SOLUTION
        return len(self._out_lists[i])
        ### END SOLUTION

In [12]:
# test for AdjacencyListDirectedGraph constructor
assert AdjacencyListDirectedGraph.__init__ != DirectedGraph.__init__, "__init__ not overridden"
test_graph = AdjacencyListDirectedGraph(10)
assert str(test_graph) == "DirectedGraph with 10 vertices and 0 edge(s):\n[]"
print("SUCCESS: AdjacencyListDirectedGraph constructor passed all tests!")

SUCCESS: AdjacencyListDirectedGraph constructor passed all tests!


In [13]:
# test for AdjacencyListDirectedGraph add_edge and has_edge
test_graph = AdjacencyListDirectedGraph(10)
test_graph.add_edge(3, 5)
assert test_graph.has_edge(3, 5)
test_graph.add_edge(2, 3)
assert test_graph.has_edge(2, 3)
assert not test_graph.has_edge(3, 2)
assert sorted(test_graph.edges()) == [(2, 3), (3, 5)]
print("SUCCESS: AdjacencyListDirectedGraph add_edge and has_edge passed all tests!")

SUCCESS: AdjacencyListDirectedGraph add_edge and has_edge passed all tests!


In [14]:
# test for AdjacencyListDirectedGraph out_edges and in_edges
assert AdjacencyListDirectedGraph.out_edges != DirectedGraph.out_edges, "out_edges not overridden"
assert AdjacencyListDirectedGraph.in_edges != DirectedGraph.in_edges, "in_edges not overridden"
test_graph = AdjacencyListDirectedGraph(10)
test_graph.add_edges([(2, 3), (2, 4), (4, 0)])
assert test_graph.out_edges(2) == [(2, 3), (2, 4)]
assert test_graph.out_edges(0) == []
assert test_graph.in_edges(0) == [(4, 0)]
assert test_graph.in_edges(2) == []
print("SUCCESS: AdjacencyListDirectedGraph out_edges and in_edges passed all tests!")

SUCCESS: AdjacencyListDirectedGraph out_edges and in_edges passed all tests!


In [15]:
# test for AdjacencyListDirectedGraph outdegree and indegree
assert AdjacencyListDirectedGraph.outdegree != DirectedGraph.outdegree, "outdegree not overridden"
assert AdjacencyListDirectedGraph.indegree != DirectedGraph.indegree, "indegree not overridden"
test_graph = AdjacencyListDirectedGraph(10)
test_graph.add_edges([(2, 3), (2, 4), (4, 0)])
assert test_graph.outdegree(2) == 2
assert test_graph.outdegree(0) == 0
assert test_graph.indegree(0) == 1
assert test_graph.indegree(2) == 0
print("SUCCESS: AdjacencyListDirectedGraph outdegree and indegree passed all tests!")

SUCCESS: AdjacencyListDirectedGraph outdegree and indegree passed all tests!
